# LLM Prototyping Notebook

Quick experiments with LLM providers

In [ ]:
# Load environment and imports
import os
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('.')

from modules.llm_provider import agent
from pydantic import BaseModel, Field
from typing import Literal, List, Optional, Union
from enum import Enum
import json
from time import time
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

In [ ]:
class CheatsheetQuestionUID(BaseModel):
    unique_question_id: int

class ResponseOption(BaseModel):
    value: Literal['Yes', 'No', 'Maybe/Unclear']
    notes: List[str] = Field(..., description="if the study...")

class CheatsheetQuestion(BaseModel):
    question_uid: CheatsheetQuestionUID
    question_formulation: str
    responses: List[ResponseOption]
    question_note: Optional[str] = Field(..., description="Any question-wide note(s), if present.")

class FinalDecision(BaseModel):
    decision: Literal['Exclude', 'Include']

class FlowDiagramStep(BaseModel):
    step_id: int
    is_last_step: bool
    on_yes: Union[CheatsheetQuestionUID, FinalDecision]
    on_maybe: Union[CheatsheetQuestionUID, FinalDecision]
    on_no: Union[CheatsheetQuestionUID, FinalDecision]

class ScreeningCheatsheet(BaseModel):
    questions: List[CheatsheetQuestion]
    flow_diagram: List[FlowDiagramStep] = Field(..., description="If you find a flow diagram in the input file, extract it. Otherwise implement the following pseudocode: { While is_last_step is False: { If Maybe/Yes Then Proceed to next step_id, Else Exclude } ; Once is_last_step is True: { If Maybe/Yes Then Include, Else Exclude } }")

print(json.dumps(ScreeningCheatsheet.model_json_schema(), indent=2, ensure_ascii=False))

In [ ]:
system_instruction = "You extract accurately data from a systematic review screening cheatsheet."

In [ ]:
# Create agent
LLMPROVIDER = "gemini"  # or "openrouter", "ollama"
llm_agent = agent(
    LLMPROVIDER,
    ScreeningCheatsheet,
    system_instruction=system_instruction,
    model_name="gemini-2.5-flash-preview-05-20")

# Test prompt
prompt = [
    "Here is the cheatsheet template:",
    f"{os.getenv("CHEATSHEET_PDF_PATH")}",  # note: docx is apparently unsupported
]

start_time = time()
response = llm_agent(prompt)
end_time = time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

In [ ]:
%%script echo skip
# Experiment with different providers
providers = ["gemini", "openrouter", "ollama"]

for provider in providers:
    try:
        print(f"\n--- Testing {provider} ---")
        test_agent = agent(provider)
        response = test_agent("What is the capital of France?")
        print(f"Response: {response.content[:100]}...")
    except Exception as e:
        print(f"Error with {provider}: {e}")